In [1]:
import sys

In [ ]:
class PartialParse(object):
    def __init__(self, sentence):
        
        self.sentence = sentence
        ###       reference the sentence object.  That is, remember to NOT modify the sentence object. 
        self.stack= ['ROOT']  # List that only has the 'ROOT' which is a default input
        self.buffer= list(sentence) 
        self.dependencies= [] 
    
    def parse_step(self, transition):
        ### YOUR CODE HERE (~7-12 Lines)
        ### TODO:
        ###     Implement a single parsing step, i.e. the logic for the following as
        ###     described in the pdf handout:
        ###         1. Shift
        ###         2. Left Arc
        ###         3. Right Arc
        if transition=='SHIFT' and self.buffer: 
            self.stack.append(self.buffer.pop(0))
        if transition=='LEFT ARC': 
            pop= self.stack.pop(-2)
            self.dependencies.append((self.stack[-2],pop))
        if transition=='RIGHT ARC':
            POP= self.stack.pop(-1)
            self.dependencies.append((self.stack[-1],POP))
        ### END YOUR CODE


def minibatch_parse(sentences, model, batch_size):
    """
    Parses a list of sentences in minibatches using a model.

    @param sentences (list of list of str): A list of sentences to be parsed (each sentence is a list of words).
    @param model (object): The model that makes parse decisions. It should have a predict method that takes a list of 
                           partial parses as input and returns a list of transitions as output.
    @param batch_size (int): The number of sentences to parse in each minibatch.

    @return (list of list of tuples): A list where each element is the list of dependencies for a parsed sentence.
    """
    # Initialize partial parses for each sentence
    partial_parses = [PartialParse(sentence) for sentence in sentences]
    # Initialize unfinished parses as a shallow copy of partial parses
    unfinished_parses = partial_parses[:]

    while unfinished_parses:
        # Take the first batch_size parses in unfinished parses as a minibatch
        minibatch = unfinished_parses[:batch_size]
        # Use the model to predict the next transition for each partial parse in the minibatch
        transitions = model.predict(minibatch)
        # Perform a parse step on each partial parse in the minibatch with its predicted transition
        for partial_parse, transition in zip(minibatch, transitions):
            partial_parse.parse_step(transition)
        # Remove the completed (empty buffer and stack of size 1) parses from unfinished parses
        unfinished_parses = [parse for parse in unfinished_parses if not (len(parse.buffer) == 0 and len(parse.stack) == 1)]

    # Return the dependencies for each (now completed) parse in partial parses
    return [partial_parse.dependencies for partial_parse in partial_parses]

# Example model for demonstration purposes
class DummyModel:
    def predict(self, partial_parses):
        # Dummy predict function that always returns "SHIFT" for simplicity
        return ["SHIFT" for _ in partial_parses]

In [ ]:
sentences = [["I", "love", "NLP"], ["This", "is", "a", "test"]]
model = DummyModel()
batch_size = 2
dependencies = minibatch_parse(sentences, model, batch_size)
print(dependencies)

In [ ]:
sentence = ["I", "love", "NLP"]
partial_parse = PartialParse(sentence)

In [19]:
print(partial_parse.stack)

['ROOT']


In [20]:
print(partial_parse.buffer)
print(partial_parse.dependencies)

['I', 'love', 'NLP']
[]


In [21]:
partial_parse.parse_step("SHIFT")
print(partial_parse.stack)       # Output: ['ROOT', 'I']
print(partial_parse.buffer)      # Output: ['love', 'NLP']

['ROOT', 'I']
['love', 'NLP']


In [22]:
partial_parse.parse_step("SHIFT")
print(partial_parse.stack)       # Output: ['ROOT', 'I', 'love']
print(partial_parse.buffer)      # Output: ['NLP']

['ROOT', 'I', 'love']
['NLP']


In [ ]:
partial_parse.parse_step("RIGHT-ARC")
print(partial_parse.stack)       # Output: ['ROOT', 'I']
print(partial_parse.buffer)      # Output: ['NLP']
print(partial_parse.dependencies) # Output: [('I', 'love')]